In [12]:
import pandas as pd
from fastai.text.all import *
from transformers import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

In [13]:
df = pd.read_csv('train.csv', error_bad_lines=False, )

/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
#Select only part of it (makes testing faster)
articles = df.head()

## Import model and set up data

In [15]:
pretrained_model_name = "facebook/bart-large"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

In [16]:
text_gen_kwargs = default_text_gen_kwargs(hf_config, hf_model, task='summarization');

In [17]:
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(
    hf_arch, hf_config, hf_tokenizer, hf_model, max_length=256, max_tgt_length=256, text_gen_kwargs=text_gen_kwargs
)
text_gen_kwargs={'max_length': 248,
 'min_length': 1,
 'do_sample': False,
 'early_stopping': True,
 'num_beams': 4,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'repetition_penalty': 1.0,
 'bad_words_ids': None,
 'bos_token_id': 0,
 'pad_token_id': 1,
 'eos_token_id': 2,
 'length_penalty': 2.0,
 'no_repeat_ngram_size': 3,
 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1,
 'decoder_start_token_id': 2,
 'use_cache': True,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': 0,
 'forced_eos_token_id': 2,
 'remove_invalid_values': False}

blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader('q'), get_y=ColReader('q\''), splitter=RandomSplitter())

In [18]:
dls = dblock.dataloaders(articles, bs=2)

/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


In [19]:
len(dls.train.items), len(dls.valid.items)

(4, 1)

### Training


In [20]:
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]
        }
    }

In [21]:
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

In [22]:
learn.fit_one_cycle(4, lr_max=3e-5, cbs=fit_cbs)

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,1.143948,2.221003,0.810811,0.742857,0.810811,0.845251,0.899624,0.871591,00:06
1,1.104637,2.221003,0.810811,0.742857,0.810811,0.845251,0.899624,0.871591,00:02
2,1.163071,2.221003,0.810811,0.742857,0.810811,0.845251,0.899624,0.871591,00:02
3,1.154799,2.174664,0.810811,0.742857,0.810811,0.845251,0.899624,0.871591,00:02


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# model2

### Generate predictions

In [18]:
grades = {
    "id": [],
    "q": [],
    "r": [],
}

In [19]:
df1 = pd.read_csv('publicpre.csv', error_bad_lines=False, )

/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:

for i in range(len(df1['q'])):
    
    outputs = learn.blurr_generate(df1['q'][i], early_stopping=False, num_return_sequences=1)
    grades["id"].append(df1.iloc[i]['id'])
    grades["q"].append('"' + outputs[0]['generated_texts'] + '"' )
        

/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/transformers/generation/utils.py:1392: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [29]:
for i in range(len(df1['q'])):
    
    outputs = learn.blurr_generate(df1['r'][i], early_stopping=False, num_return_sequences=1)
    grades["r"].append('"' + outputs[0]['generated_texts'] + '"')

/home/student/anaconda3/envs/mining/lib/python3.7/site-packages/transformers/generation/utils.py:1392: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [30]:
dfout = pd.DataFrame(grades)

In [31]:
dfout.to_csv("predict.csv",index=False)

In [32]:
print(len(grades["r"]))
print(len(grades["q"]))

2016
2016


In [33]:
dfout

,id,q,r
0,6199,""". -- -- ( Time . ) . ) -- - ( Time. ). --""","""1. If so , why do we still have apes , and why are there no"""
1,5807,"""There 's a lot of discussion there on that issue . But it really does n""","""Of course. Of course . The makers of Expelled were within their right to expel"""
2,8487,""". `` It 's not helping . The guns these people have , they do n""","""Oh , I would wager about like Mexico , about 80 % fewer than guestimated"""
3,1760,"""U.S. Shooting : 3 seriously injured in Auburn shootings | KOMO News""","""Pickup strikes group of four youths | Houston & amp ; Texas News | Chron."""
4,6228,"""4. This is the argument concerning 'choice ' that , to me , rings untrue""","""3. I believe there is a point at which we ( society ) must be prepared"""
...,...,...,...
2011,9499,"""4. You are betraying your belief system .""",""". Yep . ( I 'm assuming that by `` belief system `` , you mean"""
2012,4611,"""3. You are in a loud minority , railing against the irrational hoards about you""","""3. Being in the minority or in the majority is irrelevant . Belief has no bearing"""
2013,9328,"""I think that 'd make me happy . You bet your ass it would.""","""Well , I have two thoughts. Well , first , I probably would n't bet"""
2014,5225,""". you say `` f * * * the Constitution. ``""",""". and gun nuts say f * * * the children when we have to decide between"""
